In [1]:
from scipy.io import loadmat
import numpy as np
import scipy.optimize as opt
from pandas.io.parsers import read_csv
import sklearn.preprocessing as skp

In [2]:
def carga_csv(file_name):
    valores = read_csv(file_name, header=None).values
    return valores
#(0,1,2,3,6,7,8,9,10,11,15,17)
#(0,1,2,6,7,8,9,10,11,15,17)
#()
def seleccionAtributos(datos):
    datos = datos[:,1:].astype(float)
    datos = datos[:, (0,1,2,3,6,7,8,9,10,11,15,17)]
    return datos


datos = carga_csv('./data/bt_dataset_t3.csv')

datos = seleccionAtributos(datos)
X = datos[:,:-1]
y = datos[:, -1]
print(np.shape(X))
print(np.shape(y))

(1644, 11)
(1644,)


In [3]:
#60%
numTrain = 987
#20%
numVal = 328
#20%
numTest = 329

In [4]:
xtrain = X[:numTrain,:]
ytrain = y[:numTrain]
print(np.shape(xtrain))
print(np.shape(ytrain))

xval = X[:numVal,:]
yval = y[:numVal]
print(np.shape(xval))
print(np.shape(yval))

xtest = X[:numTest,:]
ytest = y[:numTest]
print(np.shape(xtest))
print(np.shape(ytest))

(987, 11)
(987,)
(328, 11)
(328,)
(329, 11)
(329,)


In [5]:
def mapFeature(X,p):
    poly = skp.PolynomialFeatures(p,include_bias=False)
    return poly.fit_transform(X)

In [6]:
def normalizar(X) :
    mu = np.mean(X,0)
    s = np.std(X,0)
    X = (X-mu)/s
    return (X, mu, s)

In [7]:
x = mapFeature(xtrain, 8)
(x, mu , s) = normalizar(x)
print(x.shape)

(987, 75581)


In [8]:
def despliegaMatriz(params_rn, num_entradas, num_ocultas, num_etiquetas):
    
    theta1 = np.reshape ( params_rn [:num_ocultas * (num_entradas + 1)],
    (num_ocultas, (num_entradas + 1)) )
    
    theta2 = np.reshape ( params_rn [num_ocultas * (num_entradas + 1):],
    (num_etiquetas, (num_ocultas + 1)) )
    
    return (theta1, theta2)

In [9]:
def funcion_sigmoide(z):
    return 1 / (1 + np.exp(-z))

In [10]:
def hipotesis(theta1, theta2, x):
    one = np.ones((len(x), 1))
    a1 = np.hstack([one, x])
    a2 = funcion_sigmoide(a1.dot(theta1.T))
    a2 = np.hstack([one, a2])
    a3 = funcion_sigmoide(a2.dot(theta2.T)) 
    return a3

In [11]:
def funcionCoste(theta1, theta2, x, y, num_etiquetas):

    yk = np.zeros((len(x), num_etiquetas))
    for k in range(1, num_etiquetas + 1):
        y_ = np.where(y == k, 1, 0)
        yk[:, k-1] = y_.ravel()

    return -1/len(x) * np.sum((np.log( hipotesis(theta1, theta2, x) ) * yk) 
    + (np.log( 1 - hipotesis(theta1, theta2, x) ) * (1-yk)))

In [13]:
#funcionCoste(theta1, theta2, X, y, 10)

NameError: name 'theta1' is not defined

In [14]:
def funcionCosteReg(theta1, theta2, x, y, num_etiquetas, lamb):
    
    coste = funcionCoste(theta1, theta2, x, y, num_etiquetas)
    
        
    return coste + (lamb/(2*len(x)))*(np.sum(theta1[:,1:]**2) 
    + np.sum(theta2[:, 1:]**2))

In [15]:
#funcionCosteReg(theta1, theta2, X, y, 10, 1)

NameError: name 'theta1' is not defined

In [16]:
def derivada_fun_sigmoide(z):
    return funcion_sigmoide(z) * (1 - funcion_sigmoide(z))

In [17]:
def pesosAleatorios(L_in, L_out):
    return np.random.uniform(-0.12, 0.12, (L_in, L_out))

In [18]:
def salidasCapas(theta1, theta2, x):
    a1 = np.append(1, x)
    z2 = a1.dot(theta1.T)
    a2 = funcion_sigmoide(z2)
    a2 = np.append(1, a2)
    z3 = a2.dot(theta2.T)
    a3 = funcion_sigmoide(z3) 
    
    return (a1,a2,a3,z2)

In [19]:
def gradiente(x, y, theta1, theta2, num_etiquetas) : 
 
    yk = np.zeros((len(x), num_etiquetas))
    for k in range(1, num_etiquetas + 1):
        y_ = np.where(y == k, 1, 0)
        yk[:, k-1] = y_.ravel()

    delta1 = np.zeros((len(theta1), theta1[0].size))
    delta2 = np.zeros((len(theta2), theta2[0].size))

    for i in range(0, len(x)):
        a1, a2, a3, z2 = salidasCapas(theta1, theta2, x[i,:])

        d3 = (a3 - yk[i])
        
        z2 = np.append(1, z2)
        d2 = (theta2.T).dot(d3)*derivada_fun_sigmoide(z2)
    
        d2 = np.delete(d2, 0)
        
        delta1 += np.matmul(d2[:,np.newaxis],a1[np.newaxis,:])
        delta2 += np.matmul(d3[:,np.newaxis],a2[np.newaxis,:])
        
    D1 = delta1/len(x)
    D2 = delta2/len(x)
        
    return np.concatenate((D1.ravel(), D2.ravel()))

In [20]:
def backprop(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y, reg):
    
    theta1, theta2 = despliegaMatriz(params_rn, num_entradas, num_ocultas, num_etiquetas)

    coste = funcionCoste(theta1, theta2, X, y, num_etiquetas)
    gradient = gradiente(X, y, theta1, theta2, num_etiquetas)

    
    return (coste, gradient)

In [21]:
#backprop(theta, 400, 25, 10, X, y, 1)

NameError: name 'theta' is not defined

In [22]:
def costeRN(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y, reg) :
    
    theta1, theta2 = despliegaMatriz(params_rn, num_entradas, num_ocultas, num_etiquetas)

    coste = funcionCosteReg(theta1, theta2, X, y, num_etiquetas, reg)
    grad = gradiente(X, y, theta1, theta2, num_etiquetas)
    
    delta1, delta2 = despliegaMatriz(grad, num_entradas, num_ocultas, num_etiquetas)
                
    delta1[:, 1:] += (theta1[:, 1:] * reg/len(X))
    delta2[:, 1:] += (theta2[:, 1:] * reg/len(X))
    
    gradient = np.concatenate((delta1.ravel(), delta2.ravel()))
    
    return (coste,gradient)

In [23]:
#costeRN(theta, 400, 25, 10, X, y, 1)

NameError: name 'theta' is not defined

In [26]:
def aprendizaje(lamb, iteraciones):
    thetaRnd1 = pesosAleatorios(12, 4)
    thetaRnd2 = pesosAleatorios(5, 2)
    thetaRnd = np.r_[thetaRnd1.ravel(), thetaRnd2.ravel()]
    fmin = opt.minimize(costeRN, thetaRnd, args=(11,4,2,X,y, lamb), 
                        method='TNC', options={'maxiter': iteraciones}, jac=True)
    
    optTheta = fmin.x
    theta1, theta2 = despliegaMatriz(optTheta, 11, 4, 2)
    probabilidad = hipotesis(theta1, theta2, X)
    indices  = np.argmax(probabilidad, axis=1)
    indices  += 1
    porcentaje = (indices == y.ravel())
    clasificados = porcentaje[porcentaje == True]
    percent = (len(clasificados)/len(X))*100

    print("Porcentaje de aciertos: ", percent, "%")

In [27]:
aprendizaje(1, 50)

Porcentaje de aciertos:  88.13868613138686 %


In [28]:
aprendizaje(1, 100)

Porcentaje de aciertos:  88.13868613138686 %
